In [49]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "pass"

# Connect to database via CRUD Module
shelter = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = '/home/rynewilliams_snhu/Documents/Animal Shelter/Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id = 'hidden-div', style = {'display':'none'}),
    html.Div(html.B(html.H1(['Ryne Williams - CS-340 Dashboard', 
                               html.Img(src = 'data:image/png;base64,{}'.format(encoded_image.decode()))]))),
    
    html.Hr(),
    html.Div(dcc.RadioItems(id = 'radio',
                            inline = True,
                            options = [
                                {'label': 'Water Rescue', 'value': 'W'}, 
                                {'label': 'Mountain/Wilderness Rescue', 'value': 'M'}, 
                                {'label': 'Disaster Rescue/Individual Tracking', 'value': 'D'}, 
                                {'label': 'All', 'value': 'A'}], 
                            value = 'All')),
    
    dash_table.DataTable(id = 'datatable-id',
                         columns = [{"name": i, "id": i, "deletable": False, "selectable": True} 
                                    for i in df.columns], 
                         data = df.to_dict('records'),
                         editable = False,
                         filter_action = "native",
                         sort_action = "native",
                         sort_mode = "multi",
                         column_selectable = False,
                         row_selectable = "single",
                         row_deletable = False,
                         selected_columns = [],
                         selected_rows = [0],
                         page_action = "native",
                         page_current = 0,
                         page_size = 10
                        ),
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className = 'row',
         style = {'display' : 'flex'},
             children = [html.Div(id = 'graph-id',
                                  className = 'col s12 m6',
                                 ),
                         html.Div(id = 'map-id',
                                  className = 'col s12 m6',
                                 )
                        ])
             ])


##############################################
# Interaction Between Components / Controller#
##############################################            
@app.callback(Output('datatable-id', "data"),
               Input('radio', 'value'))

def dropdown_select(value):
    df = pd.DataFrame.from_records(shelter.read({}))
               
    if (value == 'W'):
        df = pd.DataFrame.from_records(shelter.read({'$and': [{'animal_type': 'Dog'}, 
                                                              {'$or': [{'breed': 'Labrador Retriever Mix'}, 
                                                                       {'breed': 'Chesapeake Bay Retriever'}, 
                                                                       {'breed': 'Newfoundland'}]},
                                                              {'sex_upon_outcome': 'Intact Female'},
                                                              {'age_upon_outcome_in_weeks': 
                                                               {'$gte': 26, '$lte': 156}}]}))
        
    if (value == 'M'):
        df = pd.DataFrame.from_records(shelter.read({'$and': [{'animal_type': 'Dog'}, 
                                                              {'$or': [{'breed': 'German Shepard'}, 
                                                                       {'breed': 'Alaskan Malamute'}, 
                                                                       {'breed': 'Old English Sheepdog'},
                                                                       {'breed': 'Siberian Husky'},
                                                                       {'breed': 'Rottweiler'}]},
                                                              {'sex_upon_outcome': 'Intact Male'},
                                                              {'age_upon_outcome_in_weeks': 
                                                               {'$gte': 26, '$lte': 156}}]}))
        
    if (value == 'D'):
        df = pd.DataFrame.from_records(shelter.read({'$and': [{'animal_type': 'Dog'}, 
                                                              {'$or': [{'breed': 'Doberman Pinscher'}, 
                                                                       {'breed': 'German Shepard'},
                                                                       {'breed': 'Golden Retriever'},
                                                                       {'breed': 'Bloodhound'},
                                                                       {'breed': 'Rottweiler'}]},
                                                              {'sex_upon_outcome': 'Intact Male'},
                                                              {'age_upon_outcome_in_weeks': 
                                                               {'$gte': 20, '$lte': 300}}]}))
        
    if (value == 'A'):
        df = pd.DataFrame.from_records(shelter.read({}))
        
    df.drop(columns = ['_id'], inplace = True)
    return df.to_dict('records')
    
    
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(Output('graph-id', "children"),
              Input('datatable-id', "derived_virtual_data"))

def update_graphs(viewData):
    
    dff = pd.DataFrame.from_dict(viewData)

    return [
        dcc.Graph(            
            figure = px.pie(dff, names = 'breed', title = 'Preferred Animals')
        )    
    ]
    
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(Output('datatable-id', 'style_data_conditional'),
              [Input('datatable-id', 'selected_columns')])

def update_styles(selected_columns):
    
    return [{'if': { 'column_id': i },
             'background_color': '#D2F3FF'} 
            for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(Output('map-id', "children"),
              [Input('datatable-id', "derived_virtual_data"), 
               Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(viewData, index): 
    
    if viewData is None:
        return
    
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75, -97.48]
    return [dl.Map(style = {'width': '1000px', 'height': '500px'}, 
                   center = [30.75, -97.48], 
                   zoom = 10, 
                   children = [dl.TileLayer(id = "base-layer-id"),
                               # Marker with tool tip and popup
                               # Column 13 and 14 define the grid-coordinates for the map
                               # Column 4 defines the breed for the animal
                               # Column 9 defines the name of the animal
                               dl.Marker(position = [dff.iloc[row, 13], dff.iloc[row, 14]], 
                                         children = [dl.Tooltip(dff.iloc[row, 4]), 
                                                     dl.Popup([html.H1("Animal Name"), 
                                                               html.P(dff.iloc[row, 9])
                                                              ])
                                                    ])
                              ])]



app.run_server(debug = True)


Connection Successful
Dash app running on http://127.0.0.1:11255/
